In [ ]:
from google.colab import drive
drive.mount('gdrive')
import pandas as pd

Mounted at gdrive


In [ ]:
dumb_british_spellings = {"vapour":"vapor", "honour":"honor", "organisation":"organization", "harbour":"harbor", "odour":"odor", "centre":"center", "analyse":"analyze", "theatre":"theater", "colour":"color", "rumour":"rumor", "behaviour":"behavior", "armour":"armor", "grey":"gray", 
"mould":"mold", "neighbour":"neighbor", "axe":"ax", "moustache":"mustache", "plough":"plow", "mandarine":"mandarin"}
words_to_replace = {"adj":"", "gipsy":"", "albumen":"", "campanula":"", "boletus edulis":"penny bun fungus", "ursus":"", "swearword":"swear word", 'adj.':"", "coleus":"", 'n.':"", 'ОК':"ok", "typha":"cattail", "pacifica":"peaceful", "mustella":"", "smail":"", "one’s":"", "spurflower":"perennial plant", "sabre":"","equus":"", "etc.":"", "ciconia":"",  "aër":"", "panthera":"panther", "еrinaceus":"", "e.g.":"", "centaurea":"thistle", "moschiferus":"", "apterus":"", "pyrrhocoris":"", "smn.":"", "pritchardia":"", "100":"one hundred", "sabrefish":"sabre carp", "putorius":"", "adv.":"", "petromyzontidae":"", "botaurus":"", "standart":"standard", "leccinum":"", "sg.":"", "gemini":"Gemini", "tabanidae":"", "anagallis":"", "decorticate":"stiff", "albugo":"", "frangula":"", "sciurus":"", "scrofa":"", "relig.":"", "headstream":"head stream", "solanum":"", "anguilla":"", "anat.":"", "nectarinia":"", "ipomoea":"", "repaire":"repair", "vaccinium":"", "smth":"", "smth.":"", "bubo":"", "deflorate":"remove flowers", "tr.":"", "traveller":"traveler", "bubalis":"", "marmorata":"", "furuncul":"", "caballus":"", "microchiroptera":"", "urtica":"", "plumbum":"", "biol.":"", "intr.":"", "bubalus":"", "columba":"", "cucurbita":"", "goldcrest":"small bird", "melongena":"", "picea":"", "arvensis":"", "moschus":"", "psidium":"", "radiointerference":"radio interference", "owre":"", "ricinus":"", "capricorn":"goat zodiac sign", "mustela":"", "pandion":"", "adj.of":"", "nomadize":"become nomadic", "smb.":"", "kneepit":"knee pit", "num.":"", "pl.":"", "extortioner":"extortion doer", "enculturate":"assimilate", "asquint":"squint", "uliginosum":"", "heteroptera":"", "ок":"ok", "abies":"fir", "stratiotes":"", "fiddlestick":"violin bow", "scabrum":"", "grus":"bird", "acarina":"", "guajava":"", "bitterling":"freshwater fish", "lycopersicum":"", "lutra":"otter", "plectranthus":"", "macereed":"mace reed", "24":"twenty four", "acris":"", "rotundifolius":"common weed", "gutturalis":"", "oxyeleotris":"", "geometrid":"", "citrullus":"", "lepus":"", "motacilla":"", "crake":"bird", "haliaёtus":"", "glasswort":"herb", "quinsy":"throat abscess", "shoulderblade":"shoulder blade", "spearthrower":"spear thrower", "ridgepole":"ridge pole", "pimpleface":"pimple face", "tumpline":"backpack", "cushma":"clothing", "curassow":"tropical bird", "banisterium":"plant", "paca":"rodent", "netbag":"net bag", "muntjacs":"barking deer"}
#Replaces obscure words with recognizable words, replaces parts of speech with nothing, same with science words


def clean(s): # Here is the main method where we clean senses
  s = str(s)
  for i in ",:":
    s = s.replace(i, " or")
  s = s.replace("/", " or ")
  for i in "()\t\n<>?":
    s = s.replace(i, "")
  for i in "-":
    s = s.replace(i, " ")
  should_be_upper = {}
  for w in s.split(" "):
    if w != "" and w[0].isupper():
      should_be_upper = {w.lower():w.capitalize()} # Keeps track of which words should be capitalized
  s = s.lower() # Converts to lowercase
  for i in dumb_british_spellings: # Replaces British spelling with American ones
    s = s.replace(i, dumb_british_spellings[i])
  for i in words_to_replace: # Replaces all other obscure words 
    s = s.replace(i, words_to_replace[i])
  for i in should_be_upper: # Uppercases words which should be uppercase
    s = s.replace(i, should_be_upper[i])
  if len(s) > 2 and s[:3] == " or": # If the beginning word is or, that means our string was of the form "[removed word]," and we should delete the or
    s = s[3:]
  if "vs." in s: # For cases like "land (vs. sea)"" we want to convert this to "land"
    ind = s.find("vs.")
    s = s[:ind]
  s = s.replace(".","")
  return s

In [ ]:
df_raw = pd.read_csv("gdrive/MyDrive/FUSRP/filtered_dat_sem_shift.csv") 
df = df_raw.copy()
df = df.iloc[[i for i in range(len(df)) if not "<" in df.iloc[i]["meaning2"]]] # Removes pairs where sense 2 is of the form "<stuff>"
df = df.reset_index().drop(["Unnamed: 0", "realizations", "index"], axis=1)
meaning1_clean = df["meaning1"].apply(clean)
meaning2_clean = df["meaning2"].apply(clean)
df.insert(2, "meaning1_clean", meaning1_clean)
df.insert(3, "meaning2_clean", meaning2_clean)
pd.DataFrame.to_csv(df, "cleaned_dat_sem_shift.csv")